In [67]:
import numpy as np

def objective(x):
    a, b, c = x
    return (a - 0.5)**2 + (b - 15)**2 + (c - 1.5)**2  # Example function

# Bounds for each variable
bounds = np.array([[0, 1], [10, 20], [0, 3]])

# Initial guess
x0 = np.array([0.9, 19, 0.1])

def mads(func, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
    """ Mesh Adaptive Direct Search (MADS) implementation. """
    n = len(x0)
    x = x0.copy()
    delta_min = tol
    iteration = 0
    
    while delta > delta_min and iteration < max_iter:
        improved = False
        
        # Generate trial points
        for i in range(n):
            for direction in [-1, 1]:
                x_new = x.copy()
                x_new[i] += direction * delta
                x_new = np.clip(x_new, bounds[:, 0], bounds[:, 1])  # Enforce bounds
                print("x_new", i, x_new)
                if func(x_new) < func(x):
                    x = x_new
                    # delta *=2.0
                    improved = True
        
        # Reduce mesh size if no improvement
        if not improved:
            delta /= 2.0
        print(delta)
        iteration += 1
        yield x, func(x), iteration, delta

# Run MADS optimization
mads_iter = mads(objective, x0, bounds)
for x, fval, step, delta in mads_iter:
    print(f"Step {step}: x = {x}, f(x) = {fval}, delta = {delta} ")

x_new 0 [ 0.  19.   0.1]
x_new 0 [ 1.  19.   0.1]
x_new 1 [ 0.9 18.   0.1]
x_new 1 [ 0.9 19.   0.1]
x_new 2 [ 0.9 18.   0. ]
x_new 2 [ 0.9 18.   1.1]
1.0
Step 1: x = [ 0.9 18.   1.1], f(x) = 9.32, delta = 1.0 
x_new 0 [ 0.  18.   1.1]
x_new 0 [ 1.  18.   1.1]
x_new 1 [ 0.9 17.   1.1]
x_new 1 [ 0.9 18.   1.1]
x_new 2 [ 0.9 17.   0.1]
x_new 2 [ 0.9 17.   2.1]
1.0
Step 2: x = [ 0.9 17.   1.1], f(x) = 4.32, delta = 1.0 
x_new 0 [ 0.  17.   1.1]
x_new 0 [ 1.  17.   1.1]
x_new 1 [ 0.9 16.   1.1]
x_new 1 [ 0.9 17.   1.1]
x_new 2 [ 0.9 16.   0.1]
x_new 2 [ 0.9 16.   2.1]
1.0
Step 3: x = [ 0.9 16.   1.1], f(x) = 1.32, delta = 1.0 
x_new 0 [ 0.  16.   1.1]
x_new 0 [ 1.  16.   1.1]
x_new 1 [ 0.9 15.   1.1]
x_new 1 [ 0.9 16.   1.1]
x_new 2 [ 0.9 15.   0.1]
x_new 2 [ 0.9 15.   2.1]
1.0
Step 4: x = [ 0.9 15.   1.1], f(x) = 0.31999999999999995, delta = 1.0 
x_new 0 [ 0.  15.   1.1]
x_new 0 [ 1.  15.   1.1]
x_new 1 [ 0.9 14.   1.1]
x_new 1 [ 0.9 16.   1.1]
x_new 2 [ 0.9 15.   0.1]
x_new 2 [ 0.9 15.   

In [73]:
objective([ 0.9, 19.,   0.1])

18.12

In [70]:
objective([ 0.9, 19. ,  1.1])

16.32

In [ ]:
class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = [(self.x.copy(), float('inf'))]  # Store initial point
        self.waiting_for_reward = False
        self.last_trial = None
    
    def step(self, reward=None):
        """ Perform one step of MADS. Provide reward when available. """
        if self.waiting_for_reward:
            if reward is None:
                raise ValueError("Reward must be provided for the last trial step.")
            
            # Accept the trial point if the reward is better
            if reward < self.history[-1][1]:
                self.x = self.last_trial
            self.history.append((self.x.copy(), reward))
            self.waiting_for_reward = False
            
        if self.delta <= self.delta_min or self.iteration >= self.max_iter:
            return None  # Termination condition
        
        improved = False
        trial_points = []
        
        for i in range(len(self.x)):
            for direction in [-1, 1]:
                x_new = self.x.copy()
                x_new[i] += direction * self.delta
                x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                trial_points.append(x_new)
        
        self.last_trial = trial_points.pop(0)  # Pick the first trial point
        self.waiting_for_reward = True
        self.iteration += 1
        return self.last_trial

In [1]:
import numpy as np

class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = []
        self.waiting_for_reward = False
        self.last_trial = None
    
    def step(self, reward=None):
        """ Perform one step of MADS. Provide reward when available. """
        if self.waiting_for_reward:
            if reward is None:
                raise ValueError("Reward must be provided for the last trial step.")
            
            print(f"Step {self.iteration}: Received reward = {reward}")
            
            # Accept the trial point if the reward is better
            if reward < self.history[-1][1]:
                self.x = self.last_trial
            self.waiting_for_reward = False
            
        if self.delta <= self.delta_min or self.iteration >= self.max_iter:
            print(f"Final solution: x = {self.x}, f(x) = {reward}")
            return None  # Termination condition
        
        improved = False
        trial_points = []
        
        for i in range(len(self.x)):
            for direction in [-1, 1]:
                x_new = self.x.copy()
                x_new[i] += direction * self.delta
                x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                trial_points.append(x_new)
        
        self.last_trial = trial_points.pop(0)  # Pick the first trial point
        self.history.append((self.x.copy(), float('inf')))  # Placeholder reward
        self.waiting_for_reward = True
        self.iteration += 1
        
        print(f"Step {self.iteration}: Testing x = {self.last_trial}")
        return self.last_trial

# Example usage:
bounds = [(0, 1), (10, 20), (0, 3)]
x0 = [0.8, 12, 1]
mads = MADS(x0, bounds)
x_next = x0


In [2]:
def obj_func(x_next):
    return (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2

In [ ]:
mads.step(obj_func(x0))

Step 1: Testing x = [ 0. 12.  1.]


array([ 0., 12.,  1.])

In [4]:
mads.waiting_for_reward

True

In [12]:
import numpy as np

class StepwiseMADS:
    def __init__(self, x0, bounds, delta0=1.0, delta_min=1e-3, alpha=2.0, gamma=0.5):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta0
        self.delta_min = delta_min
        self.alpha = alpha
        self.gamma = gamma
        self.mesh_size = delta0
        self.iteration = 0
        self.pending_eval = None
    
    def get_next_candidate(self):
        if self.pending_eval is not None:
            raise RuntimeError("Previous evaluation result is still pending.")
        
        directions = np.eye(len(self.x))
        candidates = [self.x + self.mesh_size * d for d in directions] + [self.x - self.mesh_size * d for d in directions]
        
        candidates = [np.clip(c, self.bounds[:, 0], self.bounds[:, 1]) for c in candidates]
        self.pending_eval = candidates
        return candidates
    
    def update_with_result(self, evaluations):
        if self.pending_eval is None:
            raise RuntimeError("No pending evaluations to update with.")
        
        best_idx = np.argmin(evaluations)
        best_candidate = self.pending_eval[best_idx]
        best_value = evaluations[best_idx]
        
        if best_value < self.evaluate(self.x):
            self.x = best_candidate
            self.mesh_size *= self.alpha
        else:
            self.mesh_size *= self.gamma
        
        self.mesh_size = max(self.mesh_size, self.delta_min)
        self.pending_eval = None
        self.iteration += 1
    
    def evaluate(self, x):
        return np.sum(x**2)  # Example: minimizing distance from origin
    
    def get_current_solution(self):
        return self.x

# Example Usage
bounds = np.array([[-50, 50], [-5, 5], [-5, 5]])
mads = StepwiseMADS(x0=[30, 3, 3], bounds=bounds)

for _ in range(15):  # Run for a few iterations
    candidates = mads.get_next_candidate()
    print("candidates", candidates)
    evaluations = [np.sum(c**2) for c in candidates]  # Simulated evaluation
    mads.update_with_result(evaluations)
    print(f"Iteration {_}: Best solution {mads.get_current_solution()}")


candidates [array([31.,  3.,  3.]), array([30.,  4.,  3.]), array([30.,  3.,  4.]), array([29.,  3.,  3.]), array([30.,  2.,  3.]), array([30.,  3.,  2.])]
Iteration 0: Best solution [29.  3.  3.]
candidates [array([31.,  3.,  3.]), array([29.,  5.,  3.]), array([29.,  3.,  5.]), array([27.,  3.,  3.]), array([29.,  1.,  3.]), array([29.,  3.,  1.])]
Iteration 1: Best solution [27.  3.  3.]
candidates [array([31.,  3.,  3.]), array([27.,  5.,  3.]), array([27.,  3.,  5.]), array([23.,  3.,  3.]), array([27., -1.,  3.]), array([27.,  3., -1.])]
Iteration 2: Best solution [23.  3.  3.]
candidates [array([31.,  3.,  3.]), array([23.,  5.,  3.]), array([23.,  3.,  5.]), array([15.,  3.,  3.]), array([23., -5.,  3.]), array([23.,  3., -5.])]
Iteration 3: Best solution [15.  3.  3.]
candidates [array([31.,  3.,  3.]), array([15.,  5.,  3.]), array([15.,  3.,  5.]), array([-1.,  3.,  3.]), array([15., -5.,  3.]), array([15.,  3., -5.])]
Iteration 4: Best solution [-1.  3.  3.]
candidates [arr

In [16]:
from OMADS import POLL, SEARCH, MADS
from typing import Callable

def rosen(x):
   x = np.asarray(x)
   y = [np.sum(100.0 * (x[1:] - x[:-1] ** 2.0) ** 2.0 + (1 - x[:-1]) ** 2.0,
         axis=0), [0]]
   return y

x0 = [-2.0,-2.0]
fun: Callable = rosen
eval = {"blackbox": fun}
param = {"baseline": x0,
            "lb": [-2.5, 1.],
            "ub": [2.5, 3.],
            "var_names": ["theta", "p"],
            "scaling": [5.0, 4],
            "post_dir": "./post"}
options = {"seed": 0, "budget": 100000, "tol": 1e-12, "display": True}

data = {"evaluator": eval, "param": param, "options": options}

POLL.main(data)

 Evaluation of the starting points
End of the evaluation of the starting points
Success: fmin = [2265.25] (hmin = 9.99999999999999941e-37)
Success: fmin = [904.0] (hmin = 9.99999999999999941e-37)
"iteration=  1, bbeval=  5, psize=  [2.0, 2.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [904.0]
Success: fmin = [29.0] (hmin = 9.99999999999999941e-37)
"iteration=  2, bbeval=  9, psize=  [4.0, 4.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [29.0]
"iteration=  3, bbeval=  13, psize=  [2.0, 2.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [29.0]
Success: fmin = [25.0] (hmin = 9.99999999999999941e-37)
"iteration=  4, bbeval=  17, psize=  [4.0, 4.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [25.0]
"iteration=  5, bbeval=  21, psize=  [2.0, 2.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [25.0]
"iteration=  6, bbeval=  25, psize=  [1.0, 1.0], hmin =  1e-36, status:  FEASIBLE , fmin =  [25.0]
"iteration=  7, bbeval=  29, psize=  [0.5, 0.5], hmin =  1e-36, status:  FEASIBLE , fmin =  [25.0]


({'xmin': [1.0, 0.99999999999954525265],
  'fmin': [2.0679515313825691872e-23],
  'hmin': 9.999999999999999999e-37,
  'nbb_evals': 345,
  'niterations': 86,
  'nb_success': 26,
  'psize': [9.094947017729282e-13, 9.094947017729282e-13],
  'psuccess': [1.8189894035458565e-12, 1.8189894035458565e-12],
  'msize': [8.271806125530277e-25, 8.271806125530277e-25]},
 Dirs2n(_poll_set=[CandidatePoint(_n=2, _coords=[0.9999999999981810106, 0.99999999999954525265], _defined=[True, True], _evaluated=True, _f=[1.0166049728276710124e-21], _freal=[1.0166049728276710124e-21], _c_ineq=[0], _c_eq=[], _h=9.999999999999999999e-37, _signature=-3706400538692445471, _dtype=DType(_prec='high', _dtype=<class 'numpy.longdouble'>, _itype=<class 'numpy.int64'>, _zero=1e-18, _warned=False), _var_type=[<VAR_TYPE.REAL: 1>, <VAR_TYPE.REAL: 1>], _sets=None, _var_link=[None, None], _status=<DESIGN_STATUS.FEASIBLE: 1>, _constraints_type=[None], _is_EB_passed=True, _LAMBDA=[1e-18], _RHO=2.2323972485981934e-107, _hmax=9.999

In [134]:
class MADS:
    def __init__(self, x0, bounds, delta=1.0, tol=1e-6, max_iter=100):
        self.x = np.array(x0)
        self.bounds = np.array(bounds)
        self.delta = delta
        self.delta_min = tol
        self.max_iter = max_iter
        self.iteration = 0
        self.history = []
        self.waiting_for_reward = False
        self.last_trial = None
        self.trial_points = self.get_trial_points()
    
    def get_trial_points(self):  
        trial_points = []  
        for i in range(len(self.x)):
            for direction in [-1, 1]:
                x_new = self.x.copy()
                x_new[i] += direction * self.delta
                x_new = np.clip(x_new, self.bounds[:, 0], self.bounds[:, 1])
                trial_points.append(x_new) 
        return trial_points
    
    def step(self, reward=None):
        """ Perform one step of MADS. Provide reward when available. """
        if self.waiting_for_reward:
            if reward is None:
                raise ValueError("Reward must be provided for the last trial step.")
            
            print(f"Step {self.iteration}: Received reward = {reward}")
            self.history.append((self.x.copy(), reward)) 
            
            # Accept the trial point if the reward is better
            # print("min([i[1] for i in self.history])", min([i[1] for i in self.history]))
            if reward < min([i[1] for i in self.history]):
                self.x = self.last_trial
                

                
            self.waiting_for_reward = False
        if len(self.trial_points)>0:
            self.last_trial = self.trial_points.pop(0)
        else:
            self.trial_points = self.get_trial_points()
            self.last_trial = self.trial_points.pop(0)
            self.delta /=2
        self.waiting_for_reward = True
        self.iteration += 1
        return self.last_trial

In [135]:
# Example usage:
bounds = [(0, 10), (10, 20), (0, 3)]
x0 = [2, 12, 1]

In [136]:
mads = MADS(x0, bounds)

In [137]:
def obj_func(x_next):
    return (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2

In [138]:
x = x0
for i in range(20):
    x = mads.step(obj_func(x))
    print("x", x)
    # print("trial_points", mads.trial_points)
    # print("x", mads.x)
    # print("history", mads.history)

x [ 1 12  1]
Step 1: Received reward = 9.5
x [ 3 12  1]
Step 2: Received reward = 15.5
x [ 2 11  1]
Step 3: Received reward = 18.5
x [ 2 13  1]
Step 4: Received reward = 6.5
x [ 2 12  0]
Step 5: Received reward = 13.5
x [ 2 12  2]
Step 6: Received reward = 11.5
x [ 1 12  1]
Step 7: Received reward = 9.5
x [ 3 12  1]
Step 8: Received reward = 15.5
x [ 2 11  1]
Step 9: Received reward = 18.5
x [ 2 13  1]
Step 10: Received reward = 6.5
x [ 2 12  0]
Step 11: Received reward = 13.5
x [ 2 12  2]
Step 12: Received reward = 11.5
x [ 1 12  1]
Step 13: Received reward = 9.5
x [ 2 12  1]
Step 14: Received reward = 11.5
x [ 2 11  1]
Step 15: Received reward = 18.5
x [ 2 12  1]
Step 16: Received reward = 11.5
x [ 2 12  0]
Step 17: Received reward = 13.5
x [ 2 12  1]
Step 18: Received reward = 11.5
x [ 1 12  1]
Step 19: Received reward = 9.5
x [ 2 12  1]


In [50]:
mads.step(obj_func([4,12,1]))

Step 1: Received reward = 21.5


array([ 6, 12,  1])

In [51]:
mads.step(obj_func([ 6, 12,  1]))

Step 2: Received reward = 39.5


array([ 5, 11,  1])

In [53]:
mads.step(obj_func([ 5, 11,  1]))

Step 3: Received reward = 36.5


array([ 5, 13,  1])

In [54]:
mads.step(obj_func([ 5, 13,  1]))

Step 4: Received reward = 24.5


array([ 5, 12,  0])

In [ ]:
while True:
    x_next = mads.get_next_trial()
    print("x_next",x_next)
    if x_next is None:
        break
    
    # Simulate delayed function evaluation
    reward = (x_next[0] - 0.5)**2 + (x_next[1] - 15)**2 + (x_next[2] - 1.5)**2
    print(reward)
    mads.update_with_reward(reward)

In [1]:
a = [1,2,3]

In [2]:
a.pop()

3

In [3]:
np.eye(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [4]:
x = [ 0.9, 19.,   0. ]
mesh_size = 1
directions = np.eye(len(x))

In [5]:
candidates = [x + mesh_size * d for d in directions] + [x - mesh_size * d for d in directions]

In [55]:
candidates

[array([ 1.9, 19. ,  0. ]),
 array([ 0.9, 20. ,  0. ]),
 array([ 0.9, 19. ,  1. ]),
 array([-0.1, 19. ,  0. ]),
 array([ 0.9, 18. ,  0. ]),
 array([ 0.9, 19. , -1. ])]

In [62]:
bounds = np.array([(-3, 4), (10, 20), (0.5, 3)])

In [63]:
org_can = np.array([np.clip(c, bounds[:, 0], bounds[:, 1]) for c in candidates])

In [64]:
org_can

array([[ 1.9, 19. ,  0.5],
       [ 0.9, 20. ,  0.5],
       [ 0.9, 19. ,  1. ],
       [-0.1, 19. ,  0.5],
       [ 0.9, 18. ,  0.5],
       [ 0.9, 19. ,  0.5]])

In [47]:
org_can = np.array(candidates)

In [48]:
bounds[:, 0]

array([ -3,  10, -10])

In [49]:
org_can[:, 2]

array([ 0.,  0.,  1.,  0.,  0., -1.])

In [50]:
norm_can = []

In [51]:
for i in range(len(candidates[0])):
    norm_can.append( np.clip(org_can[:, i], bounds[i][0], bounds[i][1]) )

In [52]:
norm_can = np.array(norm_can)

In [53]:
norm_can = norm_can.T

In [54]:
norm_can

array([[ 1.9, 19. ,  0. ],
       [ 0.9, 20. ,  0. ],
       [ 0.9, 19. ,  1. ],
       [-0.1, 19. ,  0. ],
       [ 0.9, 18. ,  0. ],
       [ 0.9, 19. , -1. ]])